In [12]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from math import sqrt

# 1. Load dataset
df = pd.read_csv("Ratings.csv")
df.rename(columns={'User-ID': 'userId', 'ISBN': 'itemId', 'Book-Rating': 'rating'}, inplace=True)

# 2. Filter meaningful ratings
df = df[df['rating'] > 0]

# ✅ 3. Keep only active users and popular items
active_users = df['userId'].value_counts()[df['userId'].value_counts() > 50].index
popular_items = df['itemId'].value_counts()[df['itemId'].value_counts() > 50].index
df = df[df['userId'].isin(active_users) & df['itemId'].isin(popular_items)]

# ✅ 4. Create user-item matrix
ratings_matrix = df.pivot_table(index='userId', columns='itemId', values='rating').fillna(0)

# ✅ 5. Apply SVD
svd = TruncatedSVD(n_components=20, random_state=42)
svd_matrix = svd.fit_transform(ratings_matrix)

# ✅ 6. Predicted Ratings
predicted_ratings = np.dot(svd_matrix, svd.components_)
pred_df = pd.DataFrame(predicted_ratings, index=ratings_matrix.index, columns=ratings_matrix.columns)

# ✅ 7. Recommend Top 5 books for a specific user
user_id = ratings_matrix.index[0]
user_rated = ratings_matrix.loc[user_id][ratings_matrix.loc[user_id] > 0].index
unrated_items = [i for i in ratings_matrix.columns if i not in user_rated]
top_preds = pred_df.loc[user_id][unrated_items].sort_values(ascending=False).head(5)

print(f"\n🎯 Top 5 Recommendations for User {user_id}:\n")
for item_id, rating in top_preds.items():
    print(f"📚 Book ISBN: {item_id} — Predicted Rating: {rating:.2f}")

# ✅ 8. RMSE Calculation
true = ratings_matrix.values.flatten()
pred = pred_df.values.flatten()
rmse = sqrt(mean_squared_error(true, pred))
print(f"\n📈 RMSE: {rmse:.4f}")



🎯 Top 5 Recommendations for User 254:

📚 Book ISBN: 043935806X — Predicted Rating: 4.97
📚 Book ISBN: 059035342X — Predicted Rating: 4.14
📚 Book ISBN: 0345339681 — Predicted Rating: 2.43
📚 Book ISBN: 0385504209 — Predicted Rating: 1.76
📚 Book ISBN: 0679429220 — Predicted Rating: 1.65

📈 RMSE: 1.0693
